In [ ]:
!pip install torch
!pip3 install transformers==4.7.0 nltk==3.5 protobuf==3.15.3 sentencepiece==0.1.96


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 62.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.0 MB/s eta 0:00:00
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434694 sha256=a0e73e1b11b424dd9b3efbbae8d098a1be5a0f8a03eff07ea774bd0903d441e8
  Stored in directory: /root/.cache/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918baf

In [ ]:
!pip install -r rouge/requirements.txt
!pip install rouge-score

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=5f162f41e67d1cf9ba64cf3986b36bfacdec9b555a46daf11041616dc80d1a30
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install -r rouge/requirements.txt
!pip install rouge-score
import re 
from rouge_score import rouge_scorer

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **rouge funcation to evaluate the model **

In [ ]:
def rouge(re,summary):
  re=word_tokenize(re)
  summary=word_tokenize(summary)
  new_re=[]
  new_summary=[]
  x=1
  for i in re:
    if re.count(i)>1:
      new_re.append(str(re.index(i)))
    else:
      new_re.append(str(x))
      x=x+1
  for i in summary :
    if i in re:
      new_summary.append(str(re.index(i))) 
    else :
      new_summary.append('_') 
  x=" ".join(new_summary)
  y=" ".join(new_re)
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'])
  scores = scorer.score(x,y)
  return scores

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
m_name = "marefa-nlp/summarization-arabic-english-news"

tokenizer = AutoTokenizer.from_pretrained(m_name)
model = AutoModelWithLMHead.from_pretrained(m_name).to(device)

In [ ]:


def get_summary(text, tokenizer, model, device="cpu", num_beams=2):
    if len(text.strip()) < 50:
        return ["Please provide more longer text"]
    
    text = "summarize: <paragraph> " + " <paragraph> ".join([ s.strip() for s in sent_tokenize(text) if s.strip() != ""]) + " </s>"
    text = text.strip().replace("\n","")
    
    tokenized_text = tokenizer.encode(text, return_tensors="pt").to(device)
    
    summary_ids = model.generate(
            tokenized_text,
            max_length=512,
            num_beams=num_beams,
            repetition_penalty=1.5, 
            length_penalty=1.0, 
            early_stopping=True

    )
    
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return [ s.strip() for s in output.split("<hl>") if s.strip() != "" ]
    
## Prepare Samples

    
    


In [ ]:

samples = [
    """
  المكان جميل لاكن الاسعار عاليه جدا 
  الموظفين مش احسن حاجه 
  لا يوجد عماله مدربه
  
    """
]

## Get summaries

print("Original Article:", samples[0])
print("\n===========\nSummary: \n")
hls = get_summary(samples[0], tokenizer, model, device)
for hl in hls:
    print("\t-", hl)

Original Article: 
  المكان جميل لاكن الاسعار عاليه جدا 
  الموظفين مش احسن حاجه 
  لا يوجد عماله مدربه
  
    

Summary: 

	- وقد اتصلت بـ (ماثيو) من قبل مدير شركة (إي بي سي)


In [ ]:
from nltk.tokenize import sent_tokenize
len(sent_tokenize(samples[0]))

25

In [ ]:
hls=" ".join(hls)


In [ ]:
print(hls)

إريتريا هي دولة الحزب الواحد التي لم تجر فيها انتخابات تشريعية وطنية منذ الاستقلال. تملك أريتريا خط ساحل واسع على طول البحر الأحمر. وقد وضعت إريتريا في وضع فدرالي مع إثيوبيا لمدة 10 سنوات.


In [ ]:
re="""
إريتريا هي دولة الحزب الواحد التي لم تجر فيها انتخابات تشريعية وطنية منذ الاستقلال. تأسست مملكة أكسوم ، التي تغطي الكثير من إريتريا الحديثة وشمال إثيوبيا خلال القرن الأول أو الثاني الميلادي. إن إنشاء إريتريا المعاصرة هو نتيجة لإدماج ممالك مستقلة وسلطنة مستقلة.
"""

In [ ]:
rouge(re,hls)



INFO:absl:Using default tokenizer.


{'rouge1': Score(precision=0.41304347826086957, recall=1.0, fmeasure=0.5846153846153846),
 'rouge2': Score(precision=0.28888888888888886, recall=0.7222222222222222, fmeasure=0.4126984126984127),
 'rougeL': Score(precision=0.391304347826087, recall=0.9473684210526315, fmeasure=0.5538461538461538)}